In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
batchSize = 64
imageSize = 64

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
transform = transforms.Compose(
    [transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

In [4]:
dataset = dset.CIFAR10(root='data', download=True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)

Files already downloaded and verified


In [5]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input.cuda())
        return output

In [7]:
netG = G()
if torch.cuda.is_available():
    netG.cuda()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [8]:
class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input.cuda())
        return output.view(-1)

In [9]:
netD = D()
if torch.cuda.is_available():
    netD.cuda()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [10]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

In [11]:
for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data[0], errG.data[0]))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

c:\garage\venv\lib\site-packages\ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0/25][0/782] Loss_D: 2.1091 Loss_G: 4.6319
[0/25][1/782] Loss_D: 1.7108 Loss_G: 5.7792
[0/25][2/782] Loss_D: 1.0932 Loss_G: 6.1405
[0/25][3/782] Loss_D: 0.9081 Loss_G: 6.2842
[0/25][4/782] Loss_D: 0.9463 Loss_G: 6.8384
[0/25][5/782] Loss_D: 0.6867 Loss_G: 6.8763
[0/25][6/782] Loss_D: 0.9275 Loss_G: 7.9571
[0/25][7/782] Loss_D: 0.7534 Loss_G: 7.5670
[0/25][8/782] Loss_D: 0.7065 Loss_G: 9.1075
[0/25][9/782] Loss_D: 0.3767 Loss_G: 7.8927
[0/25][10/782] Loss_D: 1.1354 Loss_G: 10.5062
[0/25][11/782] Loss_D: 0.3400 Loss_G: 8.7097
[0/25][12/782] Loss_D: 0.9710 Loss_G: 11.3782
[0/25][13/782] Loss_D: 0.4495 Loss_G: 9.4476
[0/25][14/782] Loss_D: 0.9093 Loss_G: 13.1308
[0/25][15/782] Loss_D: 0.4817 Loss_G: 9.4372
[0/25][16/782] Loss_D: 1.0182 Loss_G: 13.8503
[0/25][17/782] Loss_D: 0.3379 Loss_G: 11.1723
[0/25][18/782] Loss_D: 0.3289 Loss_G: 9.7778
[0/25][19/782] Loss_D: 1.0200 Loss_G: 16.7548
[0/25][20/782] Loss_D: 0.3701 Loss_G: 14.2076
[0/25][21/782] Loss_D: 0.1949 Loss_G: 7.0007
[0/25][22/782

[0/25][181/782] Loss_D: 0.3214 Loss_G: 5.4933
[0/25][182/782] Loss_D: 0.3093 Loss_G: 6.0833
[0/25][183/782] Loss_D: 0.1529 Loss_G: 5.3563
[0/25][184/782] Loss_D: 0.2313 Loss_G: 6.4531
[0/25][185/782] Loss_D: 0.1746 Loss_G: 5.3574
[0/25][186/782] Loss_D: 0.2240 Loss_G: 6.2506
[0/25][187/782] Loss_D: 0.2031 Loss_G: 5.0771
[0/25][188/782] Loss_D: 0.3348 Loss_G: 7.7004
[0/25][189/782] Loss_D: 0.1973 Loss_G: 6.5429
[0/25][190/782] Loss_D: 0.1969 Loss_G: 4.3408
[0/25][191/782] Loss_D: 0.3765 Loss_G: 8.2044
[0/25][192/782] Loss_D: 0.3011 Loss_G: 6.4995
[0/25][193/782] Loss_D: 0.0735 Loss_G: 4.8492
[0/25][194/782] Loss_D: 0.2421 Loss_G: 7.4617
[0/25][195/782] Loss_D: 0.2004 Loss_G: 6.3440
[0/25][196/782] Loss_D: 0.2098 Loss_G: 5.8625
[0/25][197/782] Loss_D: 0.1968 Loss_G: 7.5412
[0/25][198/782] Loss_D: 0.3119 Loss_G: 4.8829
[0/25][199/782] Loss_D: 0.3697 Loss_G: 9.7196
[0/25][200/782] Loss_D: 0.4234 Loss_G: 6.7514
[0/25][201/782] Loss_D: 0.2673 Loss_G: 8.0654
[0/25][202/782] Loss_D: 0.1277 Los

[0/25][359/782] Loss_D: 0.5789 Loss_G: 8.1846
[0/25][360/782] Loss_D: 0.3984 Loss_G: 6.1109
[0/25][361/782] Loss_D: 0.1787 Loss_G: 3.6494
[0/25][362/782] Loss_D: 0.5480 Loss_G: 10.8374
[0/25][363/782] Loss_D: 0.6107 Loss_G: 8.8767
[0/25][364/782] Loss_D: 0.2007 Loss_G: 4.6636
[0/25][365/782] Loss_D: 0.3980 Loss_G: 7.7664
[0/25][366/782] Loss_D: 0.2609 Loss_G: 7.0146
[0/25][367/782] Loss_D: 0.1086 Loss_G: 5.3355
[0/25][368/782] Loss_D: 0.2618 Loss_G: 5.6202
[0/25][369/782] Loss_D: 0.2472 Loss_G: 4.9869
[0/25][370/782] Loss_D: 0.4096 Loss_G: 3.1899
[0/25][371/782] Loss_D: 0.5736 Loss_G: 8.5809
[0/25][372/782] Loss_D: 0.8713 Loss_G: 4.8612
[0/25][373/782] Loss_D: 0.4844 Loss_G: 5.8710
[0/25][374/782] Loss_D: 0.1491 Loss_G: 5.2474
[0/25][375/782] Loss_D: 0.2131 Loss_G: 4.3003
[0/25][376/782] Loss_D: 0.2151 Loss_G: 4.4652
[0/25][377/782] Loss_D: 0.3268 Loss_G: 5.5642
[0/25][378/782] Loss_D: 0.5913 Loss_G: 2.2961
[0/25][379/782] Loss_D: 0.5478 Loss_G: 6.4868
[0/25][380/782] Loss_D: 0.5066 Lo

[0/25][539/782] Loss_D: 0.2310 Loss_G: 4.5412
[0/25][540/782] Loss_D: 0.3443 Loss_G: 7.5817
[0/25][541/782] Loss_D: 0.7144 Loss_G: 2.9238
[0/25][542/782] Loss_D: 0.9058 Loss_G: 12.2969
[0/25][543/782] Loss_D: 0.5313 Loss_G: 11.6535
[0/25][544/782] Loss_D: 0.2074 Loss_G: 8.8055
[0/25][545/782] Loss_D: 0.1572 Loss_G: 3.7687
[0/25][546/782] Loss_D: 1.3678 Loss_G: 14.9163
[0/25][547/782] Loss_D: 1.3639 Loss_G: 11.9756
[0/25][548/782] Loss_D: 0.0621 Loss_G: 8.2670
[0/25][549/782] Loss_D: 0.2675 Loss_G: 4.6624
[0/25][550/782] Loss_D: 1.5166 Loss_G: 12.3154
[0/25][551/782] Loss_D: 1.5798 Loss_G: 7.4116
[0/25][552/782] Loss_D: 0.2920 Loss_G: 2.7308
[0/25][553/782] Loss_D: 1.2825 Loss_G: 7.6901
[0/25][554/782] Loss_D: 0.7228 Loss_G: 5.0074
[0/25][555/782] Loss_D: 0.4578 Loss_G: 4.7121
[0/25][556/782] Loss_D: 0.6412 Loss_G: 3.5112
[0/25][557/782] Loss_D: 0.8050 Loss_G: 7.2015
[0/25][558/782] Loss_D: 0.5517 Loss_G: 5.8800
[0/25][559/782] Loss_D: 1.0157 Loss_G: 2.1085
[0/25][560/782] Loss_D: 0.921

...

[23/25][695/782] Loss_D: 0.0130 Loss_G: 5.2221
[23/25][696/782] Loss_D: 0.0033 Loss_G: 7.2469
[23/25][697/782] Loss_D: 0.0241 Loss_G: 4.9462
[23/25][698/782] Loss_D: 0.0061 Loss_G: 8.8265
[23/25][699/782] Loss_D: 0.0026 Loss_G: 8.7685
[23/25][700/782] Loss_D: 0.0126 Loss_G: 8.0959
[23/25][701/782] Loss_D: 0.0074 Loss_G: 5.7658
[23/25][702/782] Loss_D: 0.0142 Loss_G: 5.2558
[23/25][703/782] Loss_D: 0.0220 Loss_G: 4.9822
[23/25][704/782] Loss_D: 0.0056 Loss_G: 6.1224
[23/25][705/782] Loss_D: 0.0127 Loss_G: 5.5392
[23/25][706/782] Loss_D: 0.0109 Loss_G: 6.5439
[23/25][707/782] Loss_D: 0.1160 Loss_G: 7.2357
[23/25][708/782] Loss_D: 0.3525 Loss_G: 8.4003
[23/25][709/782] Loss_D: 4.3058 Loss_G: 0.0683
[23/25][710/782] Loss_D: 3.4061 Loss_G: 10.0456
[23/25][711/782] Loss_D: 6.4504 Loss_G: 2.2276
[23/25][712/782] Loss_D: 0.6240 Loss_G: 0.4712
[23/25][713/782] Loss_D: 1.5063 Loss_G: 6.0958
[23/25][714/782] Loss_D: 0.6981 Loss_G: 3.9423
[23/25][715/782] Loss_D: 0.3467 Loss_G: 1.6704
[23/25][716/

[24/25][91/782] Loss_D: 0.6951 Loss_G: 1.6775
[24/25][92/782] Loss_D: 0.6038 Loss_G: 3.5094
[24/25][93/782] Loss_D: 0.5580 Loss_G: 2.6791
[24/25][94/782] Loss_D: 0.6125 Loss_G: 1.6592
[24/25][95/782] Loss_D: 0.6956 Loss_G: 3.3127
[24/25][96/782] Loss_D: 0.8882 Loss_G: 1.9181
[24/25][97/782] Loss_D: 0.6153 Loss_G: 2.2119
[24/25][98/782] Loss_D: 0.6897 Loss_G: 1.7757
[24/25][99/782] Loss_D: 0.7681 Loss_G: 4.7598
[24/25][100/782] Loss_D: 0.7662 Loss_G: 2.0633
[24/25][101/782] Loss_D: 0.5546 Loss_G: 1.1949
[24/25][102/782] Loss_D: 0.9116 Loss_G: 4.9091
[24/25][103/782] Loss_D: 0.8297 Loss_G: 2.1233
[24/25][104/782] Loss_D: 0.4427 Loss_G: 2.4298
[24/25][105/782] Loss_D: 0.6430 Loss_G: 3.8679
[24/25][106/782] Loss_D: 0.5478 Loss_G: 2.1950
[24/25][107/782] Loss_D: 0.5349 Loss_G: 2.8902
[24/25][108/782] Loss_D: 0.4199 Loss_G: 2.8186
[24/25][109/782] Loss_D: 0.4618 Loss_G: 3.2684
[24/25][110/782] Loss_D: 0.4771 Loss_G: 2.1524
[24/25][111/782] Loss_D: 0.5374 Loss_G: 3.2582
[24/25][112/782] Loss_

[24/25][267/782] Loss_D: 0.1051 Loss_G: 4.4380
[24/25][268/782] Loss_D: 0.0327 Loss_G: 4.3053
[24/25][269/782] Loss_D: 0.0434 Loss_G: 4.5748
[24/25][270/782] Loss_D: 0.0252 Loss_G: 6.3741
[24/25][271/782] Loss_D: 0.0481 Loss_G: 4.6895
[24/25][272/782] Loss_D: 0.0065 Loss_G: 6.4248
[24/25][273/782] Loss_D: 0.0410 Loss_G: 5.0043
[24/25][274/782] Loss_D: 0.0133 Loss_G: 7.0477
[24/25][275/782] Loss_D: 0.0201 Loss_G: 5.4754
[24/25][276/782] Loss_D: 0.0063 Loss_G: 6.3316
[24/25][277/782] Loss_D: 0.0600 Loss_G: 4.8901
[24/25][278/782] Loss_D: 0.0543 Loss_G: 7.3295
[24/25][279/782] Loss_D: 0.0197 Loss_G: 5.4704
[24/25][280/782] Loss_D: 0.0267 Loss_G: 4.6488
[24/25][281/782] Loss_D: 0.0121 Loss_G: 5.3918
[24/25][282/782] Loss_D: 0.0252 Loss_G: 4.9964
[24/25][283/782] Loss_D: 0.0162 Loss_G: 5.6547
[24/25][284/782] Loss_D: 0.0278 Loss_G: 4.8425
[24/25][285/782] Loss_D: 0.0160 Loss_G: 5.3086
[24/25][286/782] Loss_D: 0.0145 Loss_G: 5.2909
[24/25][287/782] Loss_D: 0.0145 Loss_G: 5.8285
[24/25][288/7

[24/25][443/782] Loss_D: 0.0231 Loss_G: 5.8629
[24/25][444/782] Loss_D: 0.0337 Loss_G: 4.7827
[24/25][445/782] Loss_D: 0.0055 Loss_G: 6.2878
[24/25][446/782] Loss_D: 0.0759 Loss_G: 4.9590
[24/25][447/782] Loss_D: 0.0059 Loss_G: 7.6013
[24/25][448/782] Loss_D: 0.0947 Loss_G: 5.9535
[24/25][449/782] Loss_D: 0.0908 Loss_G: 5.5529
[24/25][450/782] Loss_D: 0.5934 Loss_G: 4.8107
[24/25][451/782] Loss_D: 4.0668 Loss_G: 0.0115
[24/25][452/782] Loss_D: 5.4719 Loss_G: 3.0413
[24/25][453/782] Loss_D: 1.0130 Loss_G: 2.1291
[24/25][454/782] Loss_D: 0.5289 Loss_G: 2.4820
[24/25][455/782] Loss_D: 0.5491 Loss_G: 3.4959
[24/25][456/782] Loss_D: 0.6927 Loss_G: 1.3524
[24/25][457/782] Loss_D: 0.8105 Loss_G: 4.4737
[24/25][458/782] Loss_D: 0.2457 Loss_G: 4.8010
[24/25][459/782] Loss_D: 0.3917 Loss_G: 2.7129
[24/25][460/782] Loss_D: 0.8269 Loss_G: 3.6656
[24/25][461/782] Loss_D: 0.6619 Loss_G: 2.5746
[24/25][462/782] Loss_D: 0.5985 Loss_G: 3.2720
[24/25][463/782] Loss_D: 0.4766 Loss_G: 3.7167
[24/25][464/7

[24/25][619/782] Loss_D: 0.3162 Loss_G: 2.9821
[24/25][620/782] Loss_D: 0.3266 Loss_G: 3.1636
[24/25][621/782] Loss_D: 0.3900 Loss_G: 2.2187
[24/25][622/782] Loss_D: 0.5169 Loss_G: 2.9834
[24/25][623/782] Loss_D: 0.4890 Loss_G: 2.0569
[24/25][624/782] Loss_D: 0.5258 Loss_G: 3.9308
[24/25][625/782] Loss_D: 0.2912 Loss_G: 3.3809
[24/25][626/782] Loss_D: 0.1767 Loss_G: 3.0941
[24/25][627/782] Loss_D: 0.4005 Loss_G: 2.0812
[24/25][628/782] Loss_D: 0.4703 Loss_G: 4.2832
[24/25][629/782] Loss_D: 0.4499 Loss_G: 2.5220
[24/25][630/782] Loss_D: 0.3398 Loss_G: 2.8084
[24/25][631/782] Loss_D: 0.3116 Loss_G: 3.8077
[24/25][632/782] Loss_D: 0.4524 Loss_G: 1.9552
[24/25][633/782] Loss_D: 0.3852 Loss_G: 4.3764
[24/25][634/782] Loss_D: 0.4191 Loss_G: 2.3579
[24/25][635/782] Loss_D: 0.2014 Loss_G: 2.9184
[24/25][636/782] Loss_D: 0.3704 Loss_G: 4.2647
[24/25][637/782] Loss_D: 0.2682 Loss_G: 3.1448
[24/25][638/782] Loss_D: 0.3142 Loss_G: 2.3320
[24/25][639/782] Loss_D: 0.3070 Loss_G: 4.4585
[24/25][640/7